In [3]:
!pwd

/Users/gomadango/Lecture/dsprog/dsprog2/travel


###   旅館A（天然温泉　扇浜の湯　ドーミーイン川崎）の口コミ

In [9]:
import sqlite3
import os
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime

# DBファイルのパス
path = '/Users/gomadango/Lecture/dsprog/dsprog2/travel'
db_name = 'review.sqlite'
db_path = os.path.join(path, db_name)

# DBに接続する
con = sqlite3.connect(db_path)
cur = con.cursor()

# 既存のテーブルを削除
cur.execute('DROP TABLE IF EXISTS review')

# テーブルを作成するSQL文
create_table = '''CREATE TABLE IF NOT EXISTS review (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    gender TEXT,
    age TEXT,
    rating TEXT,
    hotel_name TEXT,
    scraping_date DATETIME
)'''

# テーブル作成
cur.execute(create_table)
con.commit()

# 基本URLとヘッダー情報
base_url = 'https://www.jalan.net/yad301124/kuchikomi/{page}.HTML?rootCd=041&childPriceFlg=0,0,0,0,0&roomCrack=100000&screenId=UWW3701&idx={idx}&smlCd=140102&dateUndecided=1&adultNum=1&yadNo=301124&callbackHistFlg=1&distCd=01'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_reviews(start_page=1, max_pages=10):
    page = start_page
    while page <= max_pages:
        idx = (page - 1) * 30  # ページ番号に応じたインデックス
        url = base_url.format(page=page, idx=idx)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # ホテル名を固定値「旅館A」に設定
        hotel_name = "旅館A"

        # レビュー要素の取得
        reviews = soup.find_all('div', class_='jlnpc-kuchikomiCassette')

        if not reviews:
            print(f"{page}ページ目にはレビューがありません。終了します。")
            break

        data_list = []
        for review in reviews:
            try:
                # 性別と年代の取得
                user_info = review.find('span', class_='c-label')
                if user_info:
                    user_text = user_info.text.strip()
                    gender_age = user_text.split('/')
                    gender = gender_age[0] if len(gender_age) > 0 else None
                    age = gender_age[1] if len(gender_age) > 1 else None

                # 総合評価の取得
                rating = review.find('div', class_='jlnpc-kuchikomiCassette__totalRate')
                rating = rating.text.strip() if rating else None

                # データのリストに追加
                data_list.append((gender, age, rating, hotel_name, datetime.now()))

            except Exception as e:
                print(f"エラーが発生しました: {e}")

        # データベースに挿入
        cur.executemany('INSERT INTO review (gender, age, rating, hotel_name, scraping_date) VALUES (?, ?, ?, ?, ?)', data_list)
        con.commit()

        # スリープを追加してリクエスト間隔を空ける
        time.sleep(1)

        # 次のページに進む
        page += 1

# 最初のページから最大10ページまで処理
scrape_reviews(max_pages=9)

# DBへの接続を閉じる
con.close()

print("口コミデータを取得しました。")

口コミデータを取得しました。


###   ホテルB（パールホテル川崎）の口コミ

In [11]:
# DBファイルのパス
path = '/Users/gomadango/Lecture/dsprog/dsprog2/travel'
db_name = 'review.sqlite'
db_path = os.path.join(path, db_name)

# DBに接続する
con = sqlite3.connect(db_path)
cur = con.cursor()

# URLリスト
urls = [
    'https://www.jalan.net/yad328118/kuchikomi/?screenId=UWW3001&rootCd=041&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&roomCrack=200000&childPriceFlg=0,0,0,0,0&yadNo=328118&callbackHistFlg=1&smlCd=140102&distCd=01&ccnt=lean-kuchikomi-tab'
    'https://www.jalan.net/yad328118/kuchikomi/2.HTML?childPriceFlg=0,0,0,0,0&roomCrack=200000&rootCd=041&screenId=UWW3701&idx=30&smlCd=140102&dateUndecided=1&adultNum=2&yadNo=328118&callbackHistFlg=1&distCd=01'
    ]

# ヘッダー情報
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_reviews(urls):
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # ホテル名を固定値「ホテルB」に設定
        hotel_name = "ホテルB"

        # レビュー要素の取得
        reviews = soup.find_all('div', class_='jlnpc-kuchikomiCassette')

        if not reviews:
            print(f"{url}にはレビューがありません。")
            continue

        data_list = []
        for review in reviews:
            try:
                # 性別と年代の取得
                user_info = review.find('span', class_='c-label')
                if user_info:
                    user_text = user_info.text.strip()
                    gender_age = user_text.split('/')
                    gender = gender_age[0] if len(gender_age) > 0 else None
                    age = gender_age[1] if len(gender_age) > 1 else None

                # 総合評価の取得
                rating = review.find('div', class_='jlnpc-kuchikomiCassette__totalRate')
                rating = rating.text.strip() if rating else None

                # データのリストに追加
                data_list.append((gender, age, rating, hotel_name, datetime.now()))

            except Exception as e:
                print(f"エラーが発生しました: {e}")

        # データベースに挿入
        cur.executemany('INSERT INTO review (gender, age, rating, hotel_name, scraping_date) VALUES (?, ?, ?, ?, ?)', data_list)
        con.commit()

        # スリープを追加してリクエスト間隔を空ける
        time.sleep(1)

# URLリストを処理
scrape_reviews(urls)

# DBへの接続を閉じる
con.close()

print("指定されたURLの口コミデータを取得しました。")

指定されたURLの口コミデータを取得しました。


###  ホテルC（川崎ステーションイン）の口コミ

In [13]:
# DBファイルのパス
path = '/Users/gomadango/Lecture/dsprog/dsprog2/travel'
db_name = 'review.sqlite'
db_path = os.path.join(path, db_name)

# DBに接続する
con = sqlite3.connect(db_path)
cur = con.cursor()

# URLリスト
urls = [
    'https://www.jalan.net/yad348664/kuchikomi/?screenId=UWW3001&rootCd=041&stayCount=1&roomCount=1&dateUndecided=1&adultNum=2&roomCrack=200000&childPriceFlg=0,0,0,0,0&yadNo=348664&callbackHistFlg=1&smlCd=140102&distCd=01&ccnt=lean-kuchikomi-tab',
    'https://www.jalan.net/yad348664/kuchikomi/2.HTML?rootCd=041&childPriceFlg=0,0,0,0,0&roomCrack=200000&screenId=UWW3701&idx=30&smlCd=140102&dateUndecided=1&adultNum=2&yadNo=348664&callbackHistFlg=1&distCd=01'
]

# ヘッダー情報
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_reviews(urls):
    for url in urls:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # ホテル名を固定値「ホテルC」に設定
        hotel_name = "ホテルC"

        # レビュー要素の取得
        reviews = soup.find_all('div', class_='jlnpc-kuchikomiCassette')

        if not reviews:
            print(f"{url}にはレビューがありません。")
            continue

        data_list = []
        for review in reviews:
            try:
                # 性別と年代の取得
                user_info = review.find('span', class_='c-label')
                if user_info:
                    user_text = user_info.text.strip()
                    gender_age = user_text.split('/')
                    gender = gender_age[0] if len(gender_age) > 0 else None
                    age = gender_age[1] if len(gender_age) > 1 else None

                # 総合評価の取得
                rating = review.find('div', class_='jlnpc-kuchikomiCassette__totalRate')
                rating = rating.text.strip() if rating else None

                # データのリストに追加
                data_list.append((gender, age, rating, hotel_name, datetime.now()))

            except Exception as e:
                print(f"エラーが発生しました: {e}")

        # データベースに挿入
        cur.executemany('INSERT INTO review (gender, age, rating, hotel_name, scraping_date) VALUES (?, ?, ?, ?, ?)', data_list)
        con.commit()

        # スリープを追加してリクエスト間隔を空ける
        time.sleep(1)

# URLリストを処理
scrape_reviews(urls)

# DBへの接続を閉じる
con.close()

print("指定されたURLの口コミデータを取得しました。")

指定されたURLの口コミデータを取得しました。


###   アメニティ情報
####  有→１、無→０

In [44]:
import sqlite3

# データベースの設定
db_path = 'amenities.sqlite'
con = sqlite3.connect(db_path)
cur = con.cursor()

# 既存のテーブルを削除
cur.execute('DROP TABLE IF EXISTS amenities')

# 新しいテーブルの作成
cur.execute('''
CREATE TABLE IF NOT EXISTS amenities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    hotel_name TEXT,
    hand_towel INTEGER,
    bodysoap INTEGER,
    hairdryer INTEGER,
    washlet INTEGER,
    toothbrush INTEGER,
    soap INTEGER,
    down_duvet INTEGER,
    comb_brush INTEGER,
    bath_towel INTEGER,
    yukata INTEGER,
    razor INTEGER,
    shampoo INTEGER,
    pajamas INTEGER,
    shower_cap INTEGER,
    rinse INTEGER,
    bathrobe INTEGER,
    cotton_swab INTEGER
)
''')

# データの挿入
amenities_data = [
    ("旅館A", 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0),
    ("ホテルB", 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0),
    ("ホテルC", 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1)
]

# データの挿入処理
cur.executemany('''
INSERT INTO amenities (
    hotel_name, hand_towel, bodysoap, hairdryer, washlet, toothbrush, soap, 
    down_duvet, comb_brush, bath_towel, yukata, razor, shampoo, pajamas, 
    shower_cap, rinse, bathrobe, cotton_swab
) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', amenities_data)

# コミットして接続を閉じる
con.commit()
con.close()

print("データの挿入が完了しました。")

データの挿入が完了しました。


: 